In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from utils.SN_Curve import SN_Curve_qats
from utils.create_geo_matrix import create_geo_matrix
from utils.transformations import global_2_compass, compass_2_global
import numpy as np
import sys
import pandas as pd
import os
from utils.get_scf_sector_list import get_scf_sector_list

import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = [30 / 2.54, 20 / 2.54]


## Pre-define data paths etc.

In [3]:
# Get relevant data_paths for DLC and simulation result files
data_path              = fr'{os.getcwd()}\data'
geometry_file          = data_path +  r'\DA_P53_CD.xlsx'
point_angles  = [float(i) for i in range(0,359,15)] 

# Extract file for all DEM sums per geo, per elevation
df_path= fr'{os.getcwd()}\output\python_combined_DEM.xlsx'
df_xlsx = pd.read_excel(df_path)

## Extract pre computed DEM data 

In [56]:
# import json
# with open(fr'{os.getcwd()}\output\python_combined_DEM.json') as json_file:
#     DEM_data = json.load(json_file)

geo_test_df = pd.read_excel(fr'{os.getcwd()}\unused\DA_P53_CD copy with extra geo.xlsx')
top_boat_landing_geo = geo_test_df.iloc[0]
test_elevation = top_boat_landing_geo['z']

elevations = np.array([float(key) for key in (df_xlsx['mLat'].values)])
dist = elevations - test_elevation

print(elevations)

assert (np.any(dist >= 0.)), 'Geo in question is above the highest node - invalid for interpolation'
assert (np.any(dist < 0.)), 'Geo in question is below the lowest node - invalid for interpolation'

[22.4   11.2    6.5    4.275]


In [48]:
idx_above = np.ma.MaskedArray(dist, dist < 0.0).argmin() # mask out values below 0, and find the positive value closest to zero
idx_below = np.ma.MaskedArray(dist, dist >= 0.0).argmax() # mask out values above 0, and find the negative value closest to zero
elevation_above = (elevations[idx_above])
elevation_below = (elevations[idx_below])

In [64]:
# Manually find DEM interpolated for 9.69 mLat by using manually chosen elevations
above_DEM = np.array(df_xlsx.iloc[idx_above][1:]) # remove the mLat dimension of the DataFrame
below_DEM = np.array(df_xlsx.iloc[idx_below][1:])

# Linear interpol; DEM_i = DEM_below + [dy/dx] * dx = DEM_below + [(delta DEM above/below) / (delta elevation above/below) ] * (elevation geo i - elevation below) or DEM_below + interpol_factor * delta DEM
interpol_factor = (test_elevation - elevation_below) / (elevation_above - elevation_below) 
DEM_sum = below_DEM + (above_DEM - below_DEM) * interpol_factor
verify_df = pd.DataFrame(data=np.hstack((above_DEM.reshape(-1,1), DEM_sum.reshape(-1,1), below_DEM.reshape(-1,1))), columns = ['Above DEM','Interpol DEM','Below DEM'])
# print(verify_df)

In [50]:
# Translate the angles to compass angles to align with given SCF factor
ang = [float(key) for key in df_xlsx.iloc[idx_above][1:].keys()]
compass = global_2_compass(ang)

adjusted_compass_angles, scf_sectors = get_scf_sector_list(top_boat_landing_geo, compass)

print(adjusted_compass_angles)
print(scf_sectors)

ang_back = compass_2_global(adjusted_compass_angles)
print(ang_back)


[180.0, 166.7, 150.0, 133.3, 120.0, 105.0, 90.0, 75.0, 60.0, 45.0, 30.0, 15.0, 0.0, 345.0, 330.0, 315.0, 300.0, 285.0, 270.0, 255.0, 240.0, 225.0, 210.0, 195.0]
[1.0, 1.502, 1.0, 1.502, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
[0.0, 13.300000000000011, 30.0, 46.69999999999999, 60.0, 75.0, 90.0, 105.0, 120.0, 135.0, 150.0, 165.0, 180.0, 195.0, 210.0, 225.0, 240.0, 255.0, 270.0, 285.0, 300.0, 315.0, 330.0, 345.0]


In [51]:
# Using the compass angle frame, maybe interpolate angle wise?
T_lifetime = 25.0
N_equivalent = 10.0**7
wohler_exp = 5.0
curve = SN_Curve_qats('D')
DFF = 3.0

In [68]:
DEM_tot = ((T_lifetime / N_equivalent * DEM_sum)**(1 / wohler_exp)).max()
# print(DEM_tot / 10**6)

DEM_tot_above = ((T_lifetime / N_equivalent * above_DEM)**(1 / wohler_exp)).max()
DEM_tot_below = ((T_lifetime / N_equivalent * below_DEM)**(1 / wohler_exp)).max()

print(DEM_tot / DEM_tot_above)
print(1 / DEM_tot / DEM_tot_below)

1.0135307811360146
1.0260655273854047


In [53]:
# Translate to stress
D = top_boat_landing_geo['od']
t = top_boat_landing_geo['wt'] / 1000.0
I = np.pi / 64.0 * (D**4 - (D - 2*t)**4)
S_eq = DEM_tot * D / (2 * I)
S_hs = np.multiply(S_eq, scf_sectors) # elementwise scale stress with SCF

In [55]:
print(f'Max DEM = {DEM_tot.max() / 10**6:.1f} MNm found at {adjusted_compass_angles[DEM_tot.argmax()]:.1f} degN')
print(f'Max S_eq before SCF = {S_eq.max() / 10**6:.1f} MPa found at {adjusted_compass_angles[S_eq.argmax()]:.1f} degN')
print(f'Max S_eq with SCF {S_hs.max() / 10**6:.1f} MPa found at {adjusted_compass_angles[S_hs.argmax()]:.1f} degN')
# print(f'Damage = DFF * (N * S^m / a) = {(curve.miner_sum(np.array([[S_hs.max() / 10**6, N_equivalent]])) * DFF) * 100.0 :.2f} %')

Max DEM = 97.8 MNm found at 166.7 degN
Max S_eq before SCF = 24.2 MPa found at 166.7 degN
Max S_eq with SCF 36.3 MPa found at 166.7 degN
Damage = DFF * (N * S^m / a) = 46.92 %
